<a href="https://colab.research.google.com/github/agboola01/malaria.practice/blob/master/1st_reg_sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit

In [1]:
import streamlit as st

ModuleNotFoundError: No module named 'streamlit'

# import all libraries first so that calling can be a little easier

In [1]:

import datetime,time
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(rc={'figure.figsize':(12,3)})
pd.set_option('display.max_columns', 50)
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split, KFold,cross_val_score,cross_validate,GridSearchCV
from lightgbm import LGBMRegressor,LGBMClassifier
from catboost import CatBoostRegressor,CatBoostClassifier
from xgboost import XGBRegressor,XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder,PolynomialFeatures
le=LabelEncoder()
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import SplineTransformer
plt.style.use("seaborn-whitegrid")
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm_notebook
import os

train = pd.read_csv('/content/drive/MyDrive/yusufagboola_spe/data/Train.csv')
test = pd.read_csv('/content/drive/MyDrive/yusufagboola_spe/data/Test.csv')
ss = pd.read_csv('/content/drive/MyDrive/yusufagboola_spe/data/SampleSubmission.csv')
train['ind'] = 1
test['ind'] = 0
df= pd.concat([train, test])
obj=df.select_dtypes('object')
num=df.select_dtypes(exclude='object')
df#xgr

C:\Users\agboola_yusuf\AppData\Local\Temp\ipykernel_1396\2313175460.py:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")
C:\Users\agboola_yusuf\AppData\Local\Temp\ipykernel_1396\2313175460.py:24: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/yusufagboola_spe/data/Train.csv'

In [ ]:

df['power-generated'].loc[df['power-generated'] >=32018]  = 32018
df['interaction'] = df['average-pressure-(period)'] * df['average-wind-speed-(period)']
df['sin_speed'] = np.sin(df['average-wind-speed-(period)'])
df['pressure_wind_diff'] = np.abs(df['average-pressure-(period)'] - df['average-wind-speed-(period)'])
df['time-of-day'] = abs(df['distance-to-solar-noon'] - 0.5)


In [184]:
def summary(df):
    sum = pd.DataFrame(df.dtypes, columns=['dtypes'])
    sum['missing_num'] = df.isna().sum().values*100
    sum['missing%'] = (df.isna().sum().values*100)/len(df)
    sum['uniques'] = df.nunique().values
    sum['count'] = df.count().values
    desc = pd.DataFrame(df.describe().T)
    sum['min'] = desc['min']
    sum['max'] = desc['max']
    sum['mean'] = desc['mean']
    sum['75%'] = desc['75%']
    return sum

summary(df).style.background_gradient(cmap='Blues')

,dtypes,missing_num,missing%,uniques,count,min,max,mean,75%
ID,object,0,0.000000,2920,2920,nan,nan,nan,nan
distance-to-solar-noon,float64,0,0.000000,2660,2920,0.050401,1.141361,0.503294,0.739528
temperature,int64,0,0.000000,37,2920,42.000000,78.000000,58.468493,63.000000
wind-direction,int64,0,0.000000,35,2920,1.000000,36.000000,24.953425,29.000000
wind-speed,float64,0,0.000000,159,2920,1.100000,26.600000,10.096986,13.100000
sky-cover,int64,0,0.000000,5,2920,0.000000,4.000000,1.987671,3.000000
visibility,float64,0,0.000000,17,2920,0.000000,10.000000,9.557705,10.000000
humidity,int64,0,0.000000,75,2920,14.000000,100.000000,73.513699,84.000000
average-wind-speed-(period),float64,100,0.034247,32,2919,0.000000,40.000000,10.129154,15.000000
average-pressure-(period),float64,0,0.000000,90,2920,748.792000,775.462000,762.451111,764.794000


In [ ]:

df.fillna(0,inplace=True)

In [186]:
#checking for correlations,which col to be droprd or maintain

top20_corrs = abs(df.corr()['power-generated']).sort_values(ascending = False).head(30)
top20_corrs

power-generated                1.000000e+00
distance-to-solar-noon         6.327627e-01
humidity                       4.441620e-01
time-of-day                    2.601840e-01
interaction                    2.245512e-01
average-wind-speed-(period)    2.244158e-01
pressure_wind_diff             1.928978e-01
sky-cover                      1.616399e-01
wind-direction                 1.285269e-01
wind-speed                     1.122782e-01
temperature                    1.105439e-01
visibility                     7.216032e-02
sin_speed                      3.439173e-02
average-pressure-(period)      2.984839e-02
ind                            1.743535e-16
Name: power-generated, dtype: float64

In [ ]:
# Trying to remove redundant features
def redundant_feature(df):
    redundant = []
    for i in df.columns:
        counts = df[i].value_counts()
        count_max = counts.iloc[0]
        if count_max / len(df) * 100 > 99:
            redundant.append(i)
    redundant = list(redundant)
    return redundant
redundant_features = redundant_feature(df)
redundant_features

In [ ]:
df.replace([np.inf, -np.inf], 0, inplace=True)

In [209]:
df['power-generated'].value_counts()

0.0        1799
32018.0      76
738.0         3
3.0           3
8283.0        2
           ... 
1576.0        1
4180.0        1
28515.0       1
17623.0       1
2718.0        1
Name: power-generated, Length: 1007, dtype: int64

# modelling, variation of validation and so on----

In [ ]:

col=df.drop(['ID','power-generated',],axis=1).columns

In [ ]:
train = df[df.ind == 1]
test = df[df.ind == 0]
from sklearn.model_selection  import train_test_split
X=train[col]
y = train['power-generated']


In [ ]:
cnn={'learning_rate': 0.08457828408816369, 'n_estimators': 412, 'max_depth': 8, 'l2_leaf_reg': 0.0018938792811299563, 'subsample': 0.852981565254582, 'colsample_bylevel': 0.6280580147587667, 'min_child_samples': 5}
xnn={'learning_rate': 0.1829784488366101, 'n_estimators': 75, 'max_depth': 6, 'min_child_weight': 10, 'subsample': 0.9131575093297759, 'colsample_bytree': 0.9141140004276709}
lnn= {'learning_rate': 0.09144212230116504, 'n_estimators': 104, 'max_depth': 12, 'num_leaves': 137, 'min_child_samples': 18, 'subsample': 0.6378590710393843, 'colsample_bytree': 0.9082255825084602, 'reg_lambda': 55.7510738455479, 'reg_alpha': 0.011586720501096676}


In [ ]:

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
#ctb = CatBoostRegressor(**cnn).fit(X,y)
ctb = CatBoostRegressor(**cnn).fit(X,y)
xgr = XGBRegressor(**xnn).fit(X,y)
lgr = LGBMRegressor(**lnn).fit(X,y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_predicted = ctb.predict(X_test)

RMSE = np.sqrt(mean_squared_error(y_test, (y_predicted)))

print("Root Mean Square Error:\n")
print(RMSE)#739

In [ ]:
# Analyse predictions
pred_errors = X_test.copy()
pred_errors['target'] = y_test
pred_errors['prediction'] = y_predicted
pred_errors['error'] = abs(pred_errors.prediction - pred_errors.target)
pred_errors = pred_errors[[ 'target', 'prediction', 'error']]
pred_errors.sort_values(by = 'error', ascending = False, inplace = True)
pred_errors.head()

In [ ]:
t=test[col]
ctb_pred=(ctb.predict(t))
sub = pd.DataFrame()
sub['ID'] = test['ID']
sub['power-generated']=ctb.predict(t)
sub['power-generated'].loc[sub['power-generated']<=0]  = 0
sub.to_csv("ctbcnn.csv", index=False)

lgr_pred=(ctb.predict(t))
sub = pd.DataFrame()
sub['ID'] = test['ID']
sub['power-generated']=lgr.predict(t)
sub['power-generated'].loc[sub['power-generated']<=0]  = 0
sub.to_csv("lgrt.csv", index=False)

xgr_pred=(xgr.predict(t))
sub = pd.DataFrame()
sub['ID'] = test['ID']
sub['power-generated']=xgr.predict(t)
sub['power-generated'].loc[sub['power-generated']<=0]  = 0
sub.to_csv("xgrt.csv", index=False)
sub.describe()

In [1]:
co=ctb_pred*0.4+lgr_pred*0.3+xgr_pred*0.3
sub = pd.DataFrame()
sub['ID'] = test['ID']
sub['power-generated']=(ctb_pred*0.6 + lgr_pred*0.4)
sub['power-generated'].loc[sub['power-generated']<=0]  = 0
sub.to_csv("x6_l4.csv", index=False)

NameError: name 'ctb_pred' is not defined

In [ ]:
sub['power-generated'].describe()

count      877.000000
mean      6959.556778
std       9725.317384
min          0.000000
25%         17.542208
50%       1080.203734
75%      13871.767744
max      33972.097323
Name: power-generated, dtype: float64

In [137]:

import optuna
from catboost import CatBoostRegressor
X_train, X_valid, y_train, y_valid= train_test_split(X, y, test_size=0.2, random_state=123)
# Define the objective function for Optuna
def catboost_objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 100),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 10)
    }

    model = CatBoostRegressor(**params, silent=True)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=False)
    predictions = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, predictions, squared=False)

    return rmse

# Create an Optuna study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(catboost_objective, n_trials=100)

# Print the best hyperparameters and the corresponding RMSE
best_params = study.best_params
best_rmse = study.best_value
print(f"CatBoost Best Hyperparameters: {best_params}")
print(f"CatBoost Best RMSE: {best_rmse}")


[I 2024-01-30 16:40:03,329] A new study created in memory with name: no-name-65723d9f-874a-481b-b1eb-ae533e945ae3
[I 2024-01-30 16:40:03,851] Trial 0 finished with value: 3140.037438279777 and parameters: {'learning_rate': 0.021863032930687178, 'n_estimators': 141, 'max_depth': 6, 'l2_leaf_reg': 0.2672252290925032, 'subsample': 0.7618912777293884, 'colsample_bylevel': 0.719897691829046, 'min_child_samples': 6}. Best is trial 0 with value: 3140.037438279777.
[I 2024-01-30 16:40:04,325] Trial 1 finished with value: 3067.3864380124146 and parameters: {'learning_rate': 0.05824549869535081, 'n_estimators': 177, 'max_depth': 4, 'l2_leaf_reg': 9.755781638086654e-05, 'subsample': 0.7193800513209044, 'colsample_bylevel': 0.6917000334415697, 'min_child_samples': 10}. Best is trial 1 with value: 3067.3864380124146.
[I 2024-01-30 16:40:07,409] Trial 2 finished with value: 3178.983000159501 and parameters: {'learning_rate': 0.013121825052338797, 'n_estimators': 388, 'max_depth': 8, 'l2_leaf_reg': 2

CatBoost Best Hyperparameters: {'learning_rate': 0.08457828408816369, 'n_estimators': 412, 'max_depth': 8, 'l2_leaf_reg': 0.0018938792811299563, 'subsample': 0.852981565254582, 'colsample_bylevel': 0.6280580147587667, 'min_child_samples': 5}
CatBoost Best RMSE: 2869.6934976613106


In [138]:
import optuna
import xgboost as xgb

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def xgb_objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=False)
    predictions = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, predictions, squared=False)

    return rmse

# Create an Optuna study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(xgb_objective, n_trials=100)

# Print the best hyperparameters and the corresponding RMSE
best_params = study.best_params
best_rmse = study.best_value
print(f"XGBoost Best Hyperparameters: {best_params}")
print(f"XGBoost Best RMSE: {best_rmse}")


[I 2024-01-30 16:44:25,563] A new study created in memory with name: no-name-6678d906-b048-4fac-90fd-c531654728ee
[I 2024-01-30 16:44:28,642] Trial 0 finished with value: 3335.305973720802 and parameters: {'learning_rate': 0.018845545138188195, 'n_estimators': 367, 'max_depth': 15, 'min_child_weight': 1, 'subsample': 0.9002683735014807, 'colsample_bytree': 0.8328981947607204}. Best is trial 0 with value: 3335.305973720802.
[I 2024-01-30 16:44:28,781] Trial 1 finished with value: 3070.0527120209827 and parameters: {'learning_rate': 0.15977158037222206, 'n_estimators': 368, 'max_depth': 9, 'min_child_weight': 4, 'subsample': 0.9218040437735591, 'colsample_bytree': 0.873631945313966}. Best is trial 1 with value: 3070.0527120209827.
[I 2024-01-30 16:44:29,320] Trial 2 finished with value: 3050.8532415166383 and parameters: {'learning_rate': 0.011227427343491672, 'n_estimators': 237, 'max_depth': 10, 'min_child_weight': 9, 'subsample': 0.5614236971633877, 'colsample_bytree': 0.8045865830714

XGBoost Best Hyperparameters: {'learning_rate': 0.1829784488366101, 'n_estimators': 75, 'max_depth': 6, 'min_child_weight': 10, 'subsample': 0.9131575093297759, 'colsample_bytree': 0.9141140004276709}
XGBoost Best RMSE: 2610.3660158846164


In [139]:

import lightgbm as lgb
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def lgb_objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'num_leaves': trial.suggest_int('num_leaves', 10, 200),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 100),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 100)
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train,y_train,eval_metric='rmse')
    predictions = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, predictions, squared=False)

    return rmse

# Create an Optuna study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(lgb_objective, n_trials=100)

# Print the best hyperparameters and the corresponding RMSE
best_params = study.best_params
best_rmse = study.best_value
print(f"LightGBM Best Hyperparameters: {best_params}")
print(f"LightGBM Best RMSE: {best_rmse}")

[I 2024-01-30 16:44:53,621] A new study created in memory with name: no-name-432f684b-8d3a-425d-9b42-aced4f0b76dd


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000268 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1458
[LightGBM] [Info] Number of data points in the train set: 1634, number of used features: 13
[LightGBM] [Info] Start training from score 7055.930845
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2024-01-30 16:44:54,341] Trial 0 finished with value: 2968.098007351296 and parameters: {'learning_rate': 0.08673524466678903, 'n_estimators': 401, 'max_depth': 10, 'num_leaves': 134, 'min_child_samples': 11, 'subsample': 0.9624368211302695, 'colsample_bytree': 0.9561026507177742, 'reg_lambda': 0.007907319527688758, 'reg_alpha': 0.010063343051654966}. Best is trial 0 with value: 2968.098007351296.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000255 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1458
[LightGBM] [Info] Number of data points in the train set: 1634, number of used features: 13
[LightGBM] [Info] Start training from score 7055.930845
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2024-01-30 16:44:55,278] Trial 1 finished with value: 2985.858264704256 and parameters: {'learning_rate': 0.015632374027835818, 'n_estimators': 428, 'max_depth': 8, 'num_leaves': 155, 'min_child_samples': 7, 'subsample': 0.5514840242137005, 'colsample_bytree': 0.8277433469711044, 'reg_lambda': 9.630046188869547e-05, 'reg_alpha': 0.0023045011133613212}. Best is trial 0 with value: 2968.098007351296.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000268 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1458
[LightGBM] [Info] Number of data points in the train set: 1634, number of used features: 13
[LightGBM] [Info] Start training from score 7055.930845
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[I 2024-01-30 16:45:25,888] Trial 98 finished with value: 2867.7690359902003 and parameters: {'learning_rate': 0.10876641452422636, 'n_estimators': 430, 'max_depth': 10, 'num_leaves': 145, 'min_child_samples': 18, 'subsample': 0.6048717493530138, 'colsample_bytree': 0.8923437758276491, 'reg_lambda': 2.9617803476278772, 'reg_alpha': 0.030261887551044368}. Best is trial 92 with value: 2674.48998465719.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-01-30 16:45:26,302] Trial 99 finished with value: 3132.8353500133894 and parameters: {'learning_rate': 0.03101204687728364, 'n_estimators': 79, 'max_depth': 14, 'num_leaves': 170, 'min_child_samples': 17, 'subsample': 0.6656595831800809, 'colsample_bytree': 0.8390822214585214, 'reg_lambda': 3.0546055070297085e-06, 'reg_alpha': 0.0009261501990782472}. Best is trial 92 with value: 2674.48998465719.



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
LightGBM Best Hyperparameters: {'learning_rate': 0.09144212230116504, 'n_estimators': 104, 'max_depth': 12, 'num_leaves': 137, 'min_child_samples': 18, 'subsample': 0.6378590

In [201]:
kf = KFold(n_splits=5,shuffle=False)
model = CatBoostRegressor(**cnn)
RMSE = []
R2=[]
c_train_pred = np.zeros(len(X))
c_test_pred = np.zeros(len(test))
for (train_index,test_index) in kf.split(X,y):
    X_train,X_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = y.iloc[train_index],y.iloc[test_index]

    model.fit(X_train,y_train,early_stopping_rounds=500,eval_set=[(X_test,y_test)],verbose=False)
    RMSE.append(np.sqrt(mean_squared_error(model.predict(X_test),y_test)))
    R2.append(r2_score(model.predict(X_test),y_test))

    c_train_pred+=model.predict(X)
    c_test_pred+=model.predict(test[col])
    c_predictions= np.absolute(c_test_pred/kf.n_splits)
    sub['power-generated']=c_predictions
    sub.to_csv('ctb_cv5.csv',index=False)


print(
f"r2 or accuracy:     {np.mean(R2)}\n"       #
f"Root Mean Squared Error: {np.mean(RMSE)} ")# 2896

r2 or accuracy:     0.9048165363395906
Root Mean Squared Error: 2957.089149760698 


In [199]:
import xgboost as xgb
model = xgb.XGBRegressor(**xnn)
kf = KFold(n_splits=5, shuffle=False)
RMSE = []
R2 = []
x_train_pred = np.zeros(len(X))
x_test_pred = np.zeros(len(test))

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train, early_stopping_rounds=500, eval_set=[(X_test, y_test)], verbose=False)
    predictions = model.predict(X_test)
    RMSE.append(np.sqrt(mean_squared_error(predictions, y_test)))
    R2.append(r2_score(predictions, y_test))
    x_train_pred[train_index] = model.predict(X_train)
    x_test_pred += model.predict(test[X_train.columns])
x_predictions = np.absolute(x_test_pred / kf.get_n_splits())
sub['power-generated'] = x_predictions
sub.to_csv('xgb_cv5.csv', index=False)
print(f"r2 or accuracy: {np.mean(R2)}")
print(f"Root Mean Squared Error: {np.mean(RMSE)}")


r2 or accuracy: 0.9075169938622352
Root Mean Squared Error: 2924.0531008967655


In [200]:
kf = KFold(n_splits=5,shuffle=False)
model = LGBMRegressor(**ln)
RMSE = []
R2=[]
l_train_pred = np.zeros(len(X))
l_test_pred = np.zeros(len(test))
for (train_index,test_index) in kf.split(X,y):
    X_train,X_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = y.iloc[train_index],y.iloc[test_index]

    model.fit(X_train,y_train,eval_metric='rmse')
    RMSE.append(np.sqrt(mean_squared_error(model.predict(X_test),y_test)))
    R2.append(r2_score(model.predict(X_test),y_test))

    l_train_pred+=model.predict(X)
    l_test_pred+=model.predict(test[col])
    l_predictions= np.absolute(l_test_pred/kf.n_splits)
    sub['power-generated']=l_predictions
    sub.to_csv('lgr_cv5.csv',index=False)
print(
f"r2 or accuracy:     {np.mean(R2)}\n"       #591322.5448032819
f"Root Mean Squared Error: {np.mean(RMSE)} ")#

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000312 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1454
[LightGBM] [Info] Number of data points in the train set: 1634, number of used features: 13
[LightGBM] [Info] Start training from score 6998.268054
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [ ]:
kf = KFold(n_splits=7,shuffle=False)
model = LGBMRegressor(**l_best_params)
RMSE = []
R2=[]
l_train_pred = np.zeros(len(X))
l_test_pred = np.zeros(len(test))
for (train_index,test_index) in kf.split(X,y):
    X_train,X_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = y.iloc[train_index],y.iloc[test_index]

    model.fit(X_train,y_train,eval_metric='rmse')
    RMSE.append(np.sqrt(mean_squared_error(model.predict(X_test),y_test)))
    R2.append(r2_score(model.predict(X_test),y_test))

    l_train_pred+=model.predict(X)
    l_test_pred+=model.predict(test[col])
    l_predictions= np.absolute(l_test_pred/kf.n_splits)
    ss['power-generated']=l_predictions
    ss.to_csv('lgr7cv.csv',index=False)
print(
f"r2 or accuracy:     {np.mean(R2)}\n"       #591322.5448032819
f"Root Mean Squared Error: {np.mean(RMSE)} ")#70051.88554416127

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000251 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 758
[LightGBM] [Info] Number of data points in the train set: 1634, number of used features: 11
[LightGBM] [Info] Start training from score 6998.268054
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain